In [13]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from utils import calculate_gradient_and_update, cnn_train_step, cnn_val_step, save_checkpoint, mmnn_train_step, mmnn_val_step
from dataloader import CustomDataloader
from networks import CNNRegression, MultiModalNetwork
import tqdm

#### Automatic GPU optimization, if available

In [14]:
cuda_available = torch.cuda.is_available()
print("Is CUDA available? ", cuda_available)

Is CUDA available?  True


In [15]:
if (torch.cuda.is_available()):
    device = torch.device("cuda")
    print('Using GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('GPU not available, using CPU')
    

Using GPU: NVIDIA GeForce GTX 1660 Ti


### Model 1 - Linear Regression ###

#### Load In in Val Sets

In [16]:

cm_x_train = torch.load('tensor_collection\\lr_x_train.pt')
cm_y_train = torch.load('tensor_collection\\lr_y_train.pt')
cm_x_val = torch.load('tensor_collection\\lr_x_val.pt')
cm_y_val = torch.load('tensor_collection\\lr_y_val.pt')



#### Hyperparameters setup

In [17]:
num_features = cm_x_train.shape[1]

train_loss_history = []
val_loss_history = []
num_epochs = 100
alpha = 0.1

# Initialize theta to random values between -2 and 2
theta = np.random.uniform(-2, 2, (num_features))


#### Model 1 - Training Loop 

In [18]:

# Training Loop
for t in range(num_epochs):
    # Training step
    train_loss, theta = calculate_gradient_and_update(cm_x_train, cm_y_train, theta, alpha)
    train_loss_history.append(train_loss)

    # Validation step
    val_loss, _ = calculate_gradient_and_update(cm_x_val, cm_y_val, theta, 0)  # alpha=0 to prevent updates
    val_loss_history.append(val_loss)


#### Save Model

In [19]:
theta_torch = torch.tensor(theta, dtype=torch.float32)
train_loss_history_torch = torch.tensor(train_loss_history, dtype=torch.float32)
val_loss_history_torch = torch.tensor(val_loss_history, dtype=torch.float32)

torch.save(theta_torch, 'tensor_collection\\lr_theta.pt')
torch.save(train_loss_history_torch, 'tensor_collection\\lr_train_losses.pt')
torch.save(val_loss_history_torch, 'tensor_collection\\lr_val_losses.pt')


### Model 2 - Training CNN ###

#### Load in Train, Val, and Test sets for Model 2

In [20]:
directory = 'tensor_collection\\'

cnn_x_train = pd.read_csv(directory + 'cnn_x_train.csv')
cnn_x_val = pd.read_csv(directory + 'cnn_x_val.csv')
cnn_y_train_tensor = torch.load(directory + 'cnn_y_train.pt')
cnn_y_val_tensor = torch.load(directory + 'cnn_y_val.pt')

print(type(cnn_x_train))
print(type(cnn_y_train_tensor))


<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


In [21]:
print(cnn_x_train.head())

                                            filename
0  data\images\40_0_0_20170104174309067.jpg.chip.jpg
1  data\images\26_1_2_20170116184521807.jpg.chip.jpg
2  data\images\54_0_3_20170119210113528.jpg.chip.jpg
3   data\images\6_1_0_20170114024512634.jpg.chip.jpg
4   data\images\2_1_0_20170109191758501.jpg.chip.jpg


#### Model 2 - Hyperparameters

In [22]:
model = CNNRegression().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 20
loss_fn = nn.MSELoss()
batch_sz = 100

#### Instantiating Train and Val Dataloaders

In [23]:
cnn_train_dataloader = CustomDataloader(dataframe=cnn_x_train, age=cnn_y_train_tensor, batch_size=batch_sz, randomize=True)
cnn_val_dataloader = CustomDataloader(dataframe=cnn_x_val, age=cnn_y_val_tensor, batch_size=batch_sz, randomize=False)

#### Model 2 - Training Loop

In [24]:
cnn_train_losses = []
cnn_val_losses = []
for epoch in tqdm.tqdm(range(epochs)):
    # Training step
    train_loss = cnn_train_step(model, cnn_train_dataloader, loss_fn, optimizer, device)
    cnn_train_losses.extend(train_loss)
    
    # Validation step
    val_loss = cnn_val_step(model, cnn_val_dataloader, loss_fn, device)
    cnn_val_losses.extend(val_loss)

    save_checkpoint(model, optimizer, epoch, f'model_checkpoints/cnn_checkpoint_epoch_{epoch}.pth')
save_checkpoint(model, optimizer, epochs, f'model_checkpoints/cnn_checkpoint_final.pth')

100%|██████████| 20/20 [08:50<00:00, 26.50s/it]


#### store train and validation losses

In [25]:
torch.save(cnn_train_losses, 'tensor_collection\\cnn_train_losses.pt')
torch.save(cnn_val_losses, 'tensor_collection\\cnn_val_losses.pt')

### Model 3 - Training Multi-Modal Network ###

#### Load in Train, Val, and Test sets for Model 3

In [26]:
directory = 'tensor_collection\\'


mmnn_x_train = pd.read_csv(directory + 'mmnn_x_train.csv')
mmnn_x_val = pd.read_csv(directory + 'mmnn_x_val.csv')
mmnn_y_train_tensor = torch.load(directory + 'mmnn_y_train.pt')
mmnn_y_val_tensor = torch.load(directory + 'mmnn_y_val.pt')

print(type(mmnn_x_train))
print(type(mmnn_y_train_tensor))


<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


#### Model 3 - Hyperparameters

In [27]:
num_numerical_features = mmnn_x_train.shape[1] - 1 # -1 because we don't count the filename column

model = MultiModalNetwork(num_numerical_features).to(device)  
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 20
loss_fn = torch.nn.MSELoss()
batch_sz = 200


#### Instantiating Train and Val Data loaders

In [28]:
mmnn_train_dataloader = CustomDataloader(dataframe=mmnn_x_train, age=mmnn_y_train_tensor, batch_size=batch_sz, randomize=True)
mmnn_val_dataloader = CustomDataloader(dataframe=mmnn_x_val, age=mmnn_y_val_tensor, batch_size=batch_sz, randomize=False)


#### Model 3 - Training Loop

In [29]:
mmnn_train_losses = []
mmnn_val_losses = []
for epoch in tqdm.tqdm(range(epochs)):
    # Training step
    train_loss = mmnn_train_step(model, mmnn_train_dataloader, loss_fn, optimizer, device)
    mmnn_train_losses.extend(train_loss)

    # Validation step
    val_loss = mmnn_val_step(model, mmnn_val_dataloader, loss_fn, device)
    mmnn_val_losses.extend(val_loss)

    checkpoint_filename = f'model_checkpoints/mmnn_checkpoint_epoch_{epoch}.pth'
    save_checkpoint(model, optimizer, epoch, checkpoint_filename)
save_checkpoint(model, optimizer, epochs, f'model_checkpoints/mmnn_checkpoint_final.pth')

100%|██████████| 20/20 [07:25<00:00, 22.26s/it]


#### store train and validation losses

In [30]:
torch.save(mmnn_train_losses, 'tensor_collection\\mmnn_train_losses.pt')
torch.save(mmnn_val_losses, 'tensor_collection\\mmnn_val_losses.pt')